In [ ]:
"""
Dynamic Regime-Based Sector Allocation Strategy
Eliminates Look-Ahead Bias with Walk-Forward Implementation
Rebalances every 63 trading days (~3 months)
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.mixture import GaussianMixture
from scipy.optimize import minimize
from scipy.cluster.hierarchy import dendrogram, linkage
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# ============================================================================
# 1. DATA GENERATION (Synthetic but Realistic)
# ============================================================================

class MarketDataGenerator:
    """Generate synthetic market data with realistic regime characteristics"""
    
    def __init__(self, start_date='2018-01-01', end_date='2026-01-23', seed=42):
        self.start_date = pd.to_datetime(start_date)
        self.end_date = pd.to_datetime(end_date)
        self.seed = seed
        np.random.seed(seed)
        
        self.sectors = ['XLK', 'XLF', 'XLV', 'XLE', 'XLY', 'XLP', 'XLI', 'XLB', 'XLU', 'XLRE', 'XLC']
        self.sector_names = {
            'XLK': 'Technology', 'XLF': 'Financials', 'XLV': 'Healthcare',
            'XLE': 'Energy', 'XLY': 'Consumer Discretionary', 'XLP': 'Consumer Staples',
            'XLI': 'Industrials', 'XLB': 'Materials', 'XLU': 'Utilities',
            'XLRE': 'Real Estate', 'XLC': 'Communication'
        }
        
    def generate(self):
        """Generate full market dataset"""
        dates = pd.date_range(self.start_date, self.end_date, freq='D')
        dates = dates[dates.dayofweek < 5]  # Trading days only
        
        n_days = len(dates)
        
        # Initialize price series
        prices = pd.DataFrame(index=dates, columns=self.sectors)
        for sector in self.sectors:
            prices[sector].iloc[0] = 100.0
        
        # Generate VIX and yields
        vix = pd.Series(index=dates, dtype=float)
        yield_10y = pd.Series(index=dates, dtype=float)
        yield_2y = pd.Series(index=dates, dtype=float)
        
        vix.iloc[0] = 15.0
        yield_10y.iloc[0] = 2.5
        yield_2y.iloc[0] = 2.2
        
        # Sector characteristics (drift and base volatility)
        sector_drift = {
            'XLK': 0.15/252, 'XLF': 0.10/252, 'XLV': 0.12/252,
            'XLE': 0.08/252, 'XLY': 0.13/252, 'XLP': 0.08/252,
            'XLI': 0.11/252, 'XLB': 0.09/252, 'XLU': 0.07/252,
            'XLRE': 0.09/252, 'XLC': 0.12/252
        }
        
        sector_vol = {
            'XLK': 0.20/np.sqrt(252), 'XLF': 0.18/np.sqrt(252), 'XLV': 0.15/np.sqrt(252),
            'XLE': 0.25/np.sqrt(252), 'XLY': 0.19/np.sqrt(252), 'XLP': 0.12/np.sqrt(252),
            'XLI': 0.17/np.sqrt(252), 'XLB': 0.18/np.sqrt(252), 'XLU': 0.13/np.sqrt(252),
            'XLRE': 0.16/np.sqrt(252), 'XLC': 0.18/np.sqrt(252)
        }
        
        # Simulate day by day
        current_regime = 0  # Start in bull market
        regime_true = pd.Series(index=dates, dtype=int)
        
        for i in range(1, n_days):
            date = dates[i]
            
            # Determine true regime (for visualization only, NOT used in strategy)
            # COVID crash
            if '2020-02-15' <= date.strftime('%Y-%m-%d') <= '2020-04-30':
                current_regime = 2  # Crisis
            # 2022 bear market
            elif '2022-01-01' <= date.strftime('%Y-%m-%d') <= '2022-10-31':
                current_regime = 1  # Transition
            # Normal volatility spike
            elif np.random.random() < 0.02:
                current_regime = min(2, current_regime + 1)
            # Recovery
            elif np.random.random() < 0.05 and current_regime > 0:
                current_regime = max(0, current_regime - 1)
            
            regime_true.iloc[i] = current_regime
            
            # Regime-dependent correlation
            if current_regime == 0:  # Bull
                base_correlation = 0.3
                vix_drift = -0.02
                vol_multiplier = 1.0
            elif current_regime == 1:  # Transition
                base_correlation = 0.6
                vix_drift = 0.05
                vol_multiplier = 1.5
            else:  # Crisis
                base_correlation = 0.85
                vix_drift = 0.15
                vol_multiplier = 2.5
            
            # Update VIX with mean reversion
            vix_mean = 15 if current_regime == 0 else (25 if current_regime == 1 else 45)
            vix.iloc[i] = vix.iloc[i-1] + 0.1 * (vix_mean - vix.iloc[i-1]) + np.random.normal(0, 2)
            vix.iloc[i] = max(10, min(80, vix.iloc[i]))
            
            # Update yields
            yield_10y.iloc[i] = yield_10y.iloc[i-1] + np.random.normal(0, 0.02)
            yield_2y.iloc[i] = yield_2y.iloc[i-1] + np.random.normal(0, 0.02)
            yield_10y.iloc[i] = max(0.5, min(5.0, yield_10y.iloc[i]))
            yield_2y.iloc[i] = max(0.3, min(4.5, yield_2y.iloc[i]))
            
            # Generate correlated returns for sectors
            market_shock = np.random.normal(0, 1)
            
            for sector in self.sectors:
                # Idiosyncratic component
                idio_shock = np.random.normal(0, 1)
                
                # Combined shock (correlation through market component)
                total_shock = (base_correlation * market_shock + 
                              (1 - base_correlation) * idio_shock)
                
                # Calculate return
                ret = (sector_drift[sector] + 
                       sector_vol[sector] * vol_multiplier * total_shock)
                
                # Update price
                prices[sector].iloc[i] = prices[sector].iloc[i-1] * (1 + ret)
        
        # Create combined dataframe
        data = prices.copy()
        data['VIX'] = vix
        data['YIELD_10Y'] = yield_10y
        data['YIELD_2Y'] = yield_2y
        data['TRUE_REGIME'] = regime_true  # For validation only
        
        return data


# ============================================================================
# 2. FEATURE ENGINEERING (Strict T-1 Lag)
# ============================================================================

class FeatureEngine:
    """Engineer features with NO look-ahead bias"""
    
    @staticmethod
    def engineer(data, sectors):
        """Create features using only T-1 information"""
        df = data.copy()
        
        # Calculate returns for each sector (T-1 to T)
        for sector in sectors:
            df[f'{sector}_return'] = df[sector].pct_change()
        
        # Market proxy (equal-weighted)
        df['market_return'] = df[[f'{s}_return' for s in sectors]].mean(axis=1)
        
        # CRITICAL: Shift all features by 1 to ensure T-1 data
        # Realized volatility (20-day)
        df['realized_vol_20'] = df['market_return'].rolling(20).std().shift(1) * np.sqrt(252)
        
        # Realized volatility (60-day)
        df['realized_vol_60'] = df['market_return'].rolling(60).std().shift(1) * np.sqrt(252)
        
        # VIX level (T-1)
        df['vix_level'] = df['VIX'].shift(1)
        
        # VIX change (T-2 to T-1)
        df['vix_change'] = df['VIX'].pct_change().shift(1)
        
        # Yield spread (T-1)
        df['yield_spread'] = (df['YIELD_10Y'] - df['YIELD_2Y']).shift(1)
        
        # Yield change (T-2 to T-1)
        df['yield_change'] = df['YIELD_10Y'].pct_change().shift(1)
        
        # Dispersion (cross-sectional volatility) - T-1
        sector_returns = df[[f'{s}_return' for s in sectors]]
        df['dispersion'] = sector_returns.std(axis=1).shift(1)
        
        # Drop NaN rows from rolling calculations
        df = df.dropna()
        
        return df


# ============================================================================
# 3. REGIME DETECTION (Walk-Forward, No Look-Ahead)
# ============================================================================

class RegimeDetector:
    """Detect market regimes using GMM with walk-forward methodology"""
    
    def __init__(self, n_regimes=3, min_history=252, retrain_freq=63):
        self.n_regimes = n_regimes
        self.min_history = min_history  # Need 1 year minimum
        self.retrain_freq = retrain_freq  # Retrain every 63 days
        self.feature_cols = ['realized_vol_20', 'realized_vol_60', 'vix_level', 
                            'vix_change', 'yield_spread', 'dispersion']
        
    def detect_regimes(self, features_df):
        """Walk-forward regime detection"""
        regimes = pd.Series(index=features_df.index, dtype=float)
        regime_probs = pd.DataFrame(index=features_df.index, 
                                    columns=[f'prob_regime_{i}' for i in range(self.n_regimes)])
        
        current_model = None
        regime_map = None
        
        print("Starting Walk-Forward Regime Detection...")
        
        for i in range(len(features_df)):
            date = features_df.index[i]
            
            # Skip if insufficient history
            if i < self.min_history:
                regimes.iloc[i] = np.nan
                continue
            
            # Retrain model every retrain_freq days
            if i % self.retrain_freq == 0 or current_model is None:
                # Get historical data (strictly before current date)
                train_start = max(0, i - 504)  # Use 2 years of data
                train_end = i  # Up to but not including current
                
                X_train = features_df[self.feature_cols].iloc[train_start:train_end]
                
                # Standardize features
                self.feature_mean = X_train.mean()
                self.feature_std = X_train.std()
                X_train_scaled = (X_train - self.feature_mean) / self.feature_std
                
                # Fit GMM
                gmm = GaussianMixture(
                    n_components=self.n_regimes,
                    covariance_type='full',
                    random_state=42,
                    max_iter=200,
                    n_init=10
                )
                gmm.fit(X_train_scaled)
                
                # Create stable regime mapping (sort by volatility)
                regime_vols = []
                labels = gmm.predict(X_train_scaled)
                for k in range(self.n_regimes):
                    mask = (labels == k)
                    if mask.sum() > 0:
                        regime_vols.append(X_train.loc[mask, 'realized_vol_20'].mean())
                    else:
                        regime_vols.append(0)
                
                # Map: lowest vol = 0 (Bull), highest vol = 2 (Crisis)
                sorted_indices = np.argsort(regime_vols)
                regime_map = {sorted_indices[j]: j for j in range(self.n_regimes)}
                
                current_model = gmm
                
                if i % (self.retrain_freq * 4) == 0:
                    print(f"  Retrained at {date.strftime('%Y-%m-%d')} (day {i}/{len(features_df)})")
            
            # Predict current day using model trained on data BEFORE today
            X_current = features_df[self.feature_cols].iloc[i:i+1]
            X_current_scaled = (X_current - self.feature_mean) / self.feature_std
            
            raw_regime = current_model.predict(X_current_scaled)[0]
            regimes.iloc[i] = regime_map[raw_regime]
            
            # Get probabilities
            probs = current_model.predict_proba(X_current_scaled)[0]
            for k in range(self.n_regimes):
                regime_probs.iloc[i, k] = probs[regime_map.get(k, k)]
        
        print("Regime Detection Complete!\n")
        return regimes, regime_probs


# ============================================================================
# 4. PORTFOLIO OPTIMIZATION (Historical Data Only)
# ============================================================================

class PortfolioOptimizer:
    """Optimize portfolio weights using only historical data"""
    
    def __init__(self, sectors, lookback=126):
        self.sectors = sectors
        self.lookback = lookback  # 6 months
        
    def optimize_sharpe(self, returns_hist):
        """Maximize Sharpe ratio using historical returns"""
        mean_returns = returns_hist.mean() * 252
        cov_matrix = returns_hist.cov() * 252
        
        n_assets = len(self.sectors)
        
        def neg_sharpe(weights):
            port_return = np.dot(weights, mean_returns)
            port_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
            return -port_return / (port_vol + 1e-8)
        
        constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
        bounds = tuple((0, 0.3) for _ in range(n_assets))  # Max 30% per sector
        initial = np.array([1/n_assets] * n_assets)
        
        result = minimize(neg_sharpe, initial, method='SLSQP', 
                         bounds=bounds, constraints=constraints)
        
        return dict(zip(self.sectors, result.x)) if result.success else None
    
    def optimize_min_vol(self, returns_hist):
        """Minimize volatility"""
        cov_matrix = returns_hist.cov() * 252
        n_assets = len(self.sectors)
        
        def portfolio_vol(weights):
            return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        
        constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
        bounds = tuple((0, 0.4) for _ in range(n_assets))
        initial = np.array([1/n_assets] * n_assets)
        
        result = minimize(portfolio_vol, initial, method='SLSQP',
                         bounds=bounds, constraints=constraints)
        
        return dict(zip(self.sectors, result.x)) if result.success else None
    
    def allocate_defensive(self, returns_hist):
        """Defensive allocation: Utilities, Staples, Healthcare"""
        defensive = {'XLU': 0.35, 'XLP': 0.35, 'XLV': 0.30}
        return {s: defensive.get(s, 0.0) for s in self.sectors}
    
    def allocate_cash(self):
        """Move to cash"""
        return {s: 0.0 for s in self.sectors}


# ============================================================================
# 5. BACKTESTING ENGINE
# ============================================================================

class Backtester:
    """Execute strategy with realistic timing and costs"""
    
    def __init__(self, data, features, regimes, sectors, 
                 initial_capital=100000, rebalance_freq=63, 
                 transaction_cost=0.001, strategy_type='institutional'):
        self.data = data
        self.features = features
        self.regimes = regimes
        self.sectors = sectors
        self.capital = initial_capital
        self.rebalance_freq = rebalance_freq
        self.transaction_cost = transaction_cost
        self.strategy_type = strategy_type
        
        self.optimizer = PortfolioOptimizer(sectors)
        
    def run(self):
        """Execute backtest with walk-forward rebalancing"""
        dates = self.features.index
        n_days = len(dates)
        
        # Initialize tracking
        equity_curve = pd.Series(index=dates, dtype=float)
        weights_history = pd.DataFrame(index=dates, columns=self.sectors + ['CASH'])
        trades_log = []
        
        current_weights = {s: 0 for s in self.sectors}
        current_weights['CASH'] = 1.0
        cash_position = self.capital
        
        print(f"Starting Backtest ({self.strategy_type})...")
        print(f"Rebalancing every {self.rebalance_freq} days")
        print(f"Transaction cost: {self.transaction_cost*100:.2f}%\n")
        
        for i in range(n_days):
            date = dates[i]
            
            # Mark to market
            if i == 0:
                portfolio_value = self.capital
            else:
                portfolio_value = cash_position
                for sector in self.sectors:
                    if current_weights[sector] > 0:
                        # Calculate return since last rebalance/start
                        price_today = self.data.loc[date, sector]
                        price_yesterday = self.data.loc[dates[i-1], sector]
                        sector_return = (price_today / price_yesterday) - 1
                        
                        sector_value = equity_curve.iloc[i-1] * current_weights[sector]
                        portfolio_value += sector_value * (1 + sector_return)
            
            equity_curve.iloc[i] = portfolio_value
            
            # Rebalance decision
            should_rebalance = (i % self.rebalance_freq == 0 and i > 0) or i == 0
            
            if should_rebalance and i >= 252:  # Need history for optimization
                # Get regime (T-1 to avoid look-ahead)
                regime = self.regimes.iloc[i-1] if i > 0 else 0
                
                if pd.isna(regime):
                    regime = 0
                else:
                    regime = int(regime)
                
                # Get target weights based on regime
                target_weights = self._get_target_weights(regime, date, i)
                
                if target_weights is not None:
                    # Calculate transaction costs
                    turnover = sum(abs(target_weights.get(s, 0) - current_weights.get(s, 0)) 
                                  for s in self.sectors)
                    cost = portfolio_value * turnover * self.transaction_cost
                    
                    # Execute rebalance
                    portfolio_value -= cost
                    current_weights = target_weights.copy()
                    cash_position = portfolio_value * current_weights.get('CASH', 0)
                    
                    trades_log.append({
                        'date': date,
                        'regime': regime,
                        'turnover': turnover,
                        'cost': cost,
                        'portfolio_value': portfolio_value
                    })
                    
                    if len(trades_log) <= 10:
                        print(f"  Rebalance {len(trades_log)}: {date.strftime('%Y-%m-%d')} "
                              f"| Regime {regime} | Turnover: {turnover:.2%}")
            
            # Record weights
            for sector in self.sectors:
                weights_history.loc[date, sector] = current_weights.get(sector, 0)
            weights_history.loc[date, 'CASH'] = current_weights.get('CASH', 0)
        
        print(f"\nBacktest Complete! Total Rebalances: {len(trades_log)}\n")
        
        return {
            'equity_curve': equity_curve,
            'weights_history': weights_history,
            'trades_log': pd.DataFrame(trades_log),
            'final_value': equity_curve.iloc[-1]
        }
    
    def _get_target_weights(self, regime, date, idx):
        """Determine target allocation based on regime"""
        # Get historical returns (strictly before current date)
        hist_start = max(0, idx - self.optimizer.lookback)
        hist_data = self.data.iloc[hist_start:idx]
        
        returns_hist = pd.DataFrame()
        for sector in self.sectors:
            returns_hist[sector] = hist_data[sector].pct_change()
        returns_hist = returns_hist.dropna()
        
        if len(returns_hist) < 20:
            return None
        
        # Strategy-specific allocation rules
        if self.strategy_type == 'aggressive':
            if regime == 0:  # Bull - Max Sharpe
                weights = self.optimizer.optimize_sharpe(returns_hist)
            elif regime == 1:  # Transition - Defensive
                weights = self.optimizer.allocate_defensive(returns_hist)
            else:  # Crisis - Min Vol
                weights = self.optimizer.optimize_min_vol(returns_hist)
        
        elif self.strategy_type == 'institutional':
            if regime == 0:  # Bull - Max Sharpe
                weights = self.optimizer.optimize_sharpe(returns_hist)
            elif regime == 1:  # Transition - Min Vol
                weights = self.optimizer.optimize_min_vol(returns_hist)
            else:  # Crisis - Cash
                weights = self.optimizer.allocate_cash()
        
        else:  # 'conservative'
            if regime == 0:  # Bull - Min Vol
                weights = self.optimizer.optimize_min_vol(returns_hist)
            elif regime == 1:  # Transition - Defensive
                weights = self.optimizer.allocate_defensive(returns_hist)
            else:  # Crisis - Cash
                weights = self.optimizer.allocate_cash()
        
        if weights is None:
            return None
        
        # Add cash position
        equity_weight = sum(weights.values())
        weights['CASH'] = max(0, 1 - equity_weight)
        
        return weights


# ============================================================================
# 6. PERFORMANCE METRICS
# ============================================================================

class PerformanceAnalyzer:
    """Calculate comprehensive performance metrics"""
    
    @staticmethod
    def calculate_metrics(equity_curve, benchmark_curve=None):
        """Calculate all performance metrics"""
        returns = equity_curve.pct_change().dropna()
        
        total_return = (equity_curve.iloc[-1] / equity_curve.iloc[0]) - 1
        
        # Annualized metrics
        n_years = len(equity_curve) / 252
        cagr = (equity_curve.iloc[-1] / equity_curve.iloc[0]) ** (1/n_years) - 1
        
        annual_vol = returns.std() * np.sqrt(252)
        sharpe = (cagr - 0.02) / annual_vol if annual_vol > 0 else 0
        
        # Downside metrics
        downside_returns = returns[returns < 0]
        downside_vol = downside_returns.std() * np.sqrt(252)
        sortino = (cagr - 0.02) / downside_vol if downside_vol > 0 else 0
        
        # Drawdown
        cumulative = (1 + returns).cumprod()
        running_max = cumulative.expanding().max()
        drawdown = (cumulative - running_max) / running_max
        max_drawdown = drawdown.min()
        
        calmar = cagr / abs(max_drawdown) if max_drawdown != 0 else 0
        
        metrics = {
            'Total Return': total_return,
            'CAGR': cagr,
            'Annual Volatility': annual_vol,
            'Sharpe Ratio': sharpe,
            'Sortino Ratio': sortino,
            'Max Drawdown': max_drawdown,
            'Calmar Ratio': calmar
        }
        
        # Beta and alpha if benchmark provided
        if benchmark_curve is not None:
            bench_returns = benchmark_curve.pct_change().dropna()
            aligned_returns = returns.align(bench_returns, join='inner')
            
            covariance = np.cov(aligned_returns[0], aligned_returns[1])[0, 1]
            bench_variance = aligned_returns[1].var()
            beta = covariance / bench_variance if bench_variance > 0 else 1
            
            bench_cagr = (benchmark_curve.iloc[-1] / benchmark_curve.iloc[0]) ** (1/n_years) - 1
            alpha = cagr - (0.02 + beta * (bench_cagr - 0.02))
            
            metrics['Beta'] = beta
            metrics['Alpha'] = alpha
        
        return metrics, drawdown


# ============================================================================
# 7. VISUALIZATION
# ============================================================================

class Visualizer:
    """Create comprehensive visualizations"""
    
    @staticmethod
    def plot_regime_classification(data, regimes, save_path=None):
        """Plot market with regime coloring"""
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
        
        # Calculate market index (equal-weighted)
        sectors = ['XLK', 'XLF', 'XLV', 'XLE', 'XLY', 'XLP', 'XLI', 'XLB', 'XLU', 'XLRE', 'XLC']
        market = data[sectors].mean(axis=1)
        
        # Plot 1: Market with regime colors
        regime_colors = {0: 'green', 1: 'yellow', 2: 'red'}
        
        for regime in [0, 1, 2]:
            mask = (regimes == regime)
            if mask.sum() > 0:
                ax1.scatter(regimes[mask].index, market[mask], 
                          c=regime_colors[regime], s=1, alpha=0.6,
                          label=f'Regime {regime}')
        
        ax1.set_title('Market Performance with Detected Regimes', fontsize=14, fontweight='bold')
        ax1.set_ylabel('Market Index', fontsize=12)
        ax1.legend(['Bull (0)', 'Transition (1)', 'Crisis (2)'], loc='upper left')
        ax1.grid(alpha=0.3)
        
        # Plot 2: VIX with regime shading
        ax2.plot(data.index, data['VIX'], color='black', linewidth=1, alpha=0.7)
        
        for regime in [0, 1, 2]:
            mask = (regimes == regime)
            if mask.sum() > 0:
                ax2.fill_between(regimes[mask].index, 0, 80, 
                                color=regime_colors[regime], alpha=0.2)
        
        ax2.set_title('VIX (Implied Volatility) with Regime Overlay', fontsize=14, fontweight='bold')
        ax2.set_ylabel('VIX Level', fontsize=12)
        ax2.set_xlabel('Date', fontsize=12)
        ax2.grid(alpha=0.3)
        
        plt.tight_layout()
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()
    
    @staticmethod
    def plot_correlation_analysis(data, regimes, sectors):
        """Plot correlation heatmaps by regime"""
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        regime_names = ['Bull Market', 'Transition', 'Crisis']
        
        for regime_idx, ax in enumerate(axes):
            mask = (regimes == regime_idx)
            
            if mask.sum() < 20:
                continue
            
            # Calculate correlation matrix for this regime
            regime_data = data.loc[mask, sectors].pct_change().dropna()
            corr_matrix = regime_data.corr()
            
            # Plot heatmap
            sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdYlBu_r',
                       center=0, vmin=-0.2, vmax=1.0, ax=ax, cbar_kws={'label': 'Correlation'})
            ax.set_title(f'{regime_names[regime_idx]}\n(n={mask.sum()} days)', 
                        fontsize=12, fontweight='bold')
            
            # Rotate labels
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
            ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
        
        plt.suptitle('Sector Correlation Breakdown Across Regimes', 
                    fontsize=14, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.show()
    
    @staticmethod
    def plot_hierarchical_clustering(data, regimes, sectors):
        """Plot dendrograms for Bull vs Crisis"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Bull market
        bull_mask = (regimes == 0)
        if bull_mask.sum() > 50:
            bull_data = data.loc[bull_mask, sectors].pct_change().dropna()
            bull_corr = bull_data.corr()
            
            linkage_matrix = linkage(bull_corr, method='ward')
            dendrogram(linkage_matrix, labels=sectors, ax=ax1, leaf_font_size=10)
            ax1.set_title('Bull Market (Regime 0)\nStructural Diversity', 
                         fontsize=12, fontweight='bold')
            ax1.set_ylabel('Distance', fontsize=11)
        
        # Crisis market
        crisis_mask = (regimes == 2)
        if crisis_mask.sum() > 50:
            crisis_data = data.loc[crisis_mask, sectors].pct_change().dropna()
            crisis_corr = crisis_data.corr()
            
            linkage_matrix = linkage(crisis_corr, method='ward')
            dendrogram(linkage_matrix, labels=sectors, ax=ax2, leaf_font_size=10)
            ax2.set_title('Crisis Market (Regime 2)\nCollapse of Diversification', 
                         fontsize=12, fontweight='bold')
            ax2.set_ylabel('Distance', fontsize=11)
        
        plt.suptitle('Hierarchical Clustering: The Failure of Diversification', 
                    fontsize=14, fontweight='bold', y=1.00)
        plt.tight_layout()
        plt.show()
    
    @staticmethod
    def plot_equity_curves(results_dict, benchmark_curve):
        """Plot strategy comparison"""
        fig, ax = plt.subplots(figsize=(14, 7))
        
        # Plot benchmark
        benchmark_normalized = benchmark_curve / benchmark_curve.iloc[0] * 100
        ax.plot(benchmark_curve.index, benchmark_normalized, 
               label='S&P 500 (Benchmark)', linewidth=2.5, color='gray', alpha=0.7)
        
        # Plot strategies
        colors = ['green', 'blue', 'orange']
        for (name, results), color in zip(results_dict.items(), colors):
            equity = results['equity_curve']
            normalized = equity / equity.iloc[0] * 100
            ax.plot(equity.index, normalized, label=name, linewidth=2, color=color)
        
        ax.set_title('Strategy Performance Comparison (2018-2026)', 
                    fontsize=14, fontweight='bold')
        ax.set_ylabel('Portfolio Value (Base = 100)', fontsize=12)
        ax.set_xlabel('Date', fontsize=12)
        ax.legend(loc='upper left', fontsize=11)
        ax.grid(alpha=0.3)
        ax.set_yscale('log')
        
        plt.tight_layout()
        plt.show()
    
    @staticmethod
    def plot_drawdowns(results_dict, benchmark_curve):
        """Plot drawdown comparison"""
        fig, ax = plt.subplots(figsize=(14, 6))
        
        # Calculate benchmark drawdown
        bench_returns = benchmark_curve.pct_change()
        bench_cum = (1 + bench_returns).cumprod()
        bench_max = bench_cum.expanding().max()
        bench_dd = (bench_cum - bench_max) / bench_max
        
        ax.fill_between(bench_dd.index, bench_dd * 100, 0, 
                       alpha=0.3, color='gray', label='S&P 500')
        
        colors = ['green', 'blue', 'orange']
        for (name, results), color in zip(results_dict.items(), colors):
            equity = results['equity_curve']
            returns = equity.pct_change()
            cumulative = (1 + returns).cumprod()
            running_max = cumulative.expanding().max()
            drawdown = (cumulative - running_max) / running_max
            
            ax.plot(drawdown.index, drawdown * 100, label=name, 
                   linewidth=2, color=color)
        
        ax.set_title('Drawdown Comparison: Capital Preservation', 
                    fontsize=14, fontweight='bold')
        ax.set_ylabel('Drawdown (%)', fontsize=12)
        ax.set_xlabel('Date', fontsize=12)
        ax.legend(loc='lower left', fontsize=11)
        ax.grid(alpha=0.3)
        ax.axhline(y=0, color='black', linestyle='--', linewidth=1)
        
        plt.tight_layout()
        plt.show()
    
    @staticmethod
    def plot_weights_evolution(weights_history, regimes):
        """Plot portfolio weights over time"""
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
        
        # Stacked area chart of sector weights
        sectors = [col for col in weights_history.columns if col != 'CASH']
        ax1.stackplot(weights_history.index, 
                     *[weights_history[s].fillna(0) * 100 for s in sectors],
                     labels=sectors, alpha=0.7)
        
        ax1.set_title('Portfolio Sector Allocation Over Time', 
                     fontsize=14, fontweight='bold')
        ax1.set_ylabel('Weight (%)', fontsize=12)
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=9)
        ax1.grid(alpha=0.3)
        ax1.set_ylim(0, 100)
        
        # Cash allocation with regime overlay
        regime_colors = {0: 'green', 1: 'yellow', 2: 'red'}
        
        for regime in [0, 1, 2]:
            mask = (regimes == regime)
            if mask.sum() > 0:
                ax2.fill_between(regimes[mask].index, 0, 100,
                               color=regime_colors[regime], alpha=0.2)
        
        ax2.plot(weights_history.index, weights_history['CASH'] * 100, 
                color='black', linewidth=2, label='Cash Weight')
        ax2.set_title('Cash Allocation vs Market Regime', 
                     fontsize=14, fontweight='bold')
        ax2.set_ylabel('Cash Weight (%)', fontsize=12)
        ax2.set_xlabel('Date', fontsize=12)
        ax2.legend(loc='upper right', fontsize=11)
        ax2.grid(alpha=0.3)
        ax2.set_ylim(0, 105)
        
        plt.tight_layout()
        plt.show()
    
    @staticmethod
    def plot_risk_reward(metrics_dict):
        """Scatter plot of risk vs return"""
        fig, ax = plt.subplots(figsize=(10, 7))
        
        for name, metrics in metrics_dict.items():
            ax.scatter(metrics['Annual Volatility'] * 100, 
                      metrics['CAGR'] * 100,
                      s=200, alpha=0.7, label=name)
            
            # Add labels
            ax.annotate(name, 
                       (metrics['Annual Volatility'] * 100, metrics['CAGR'] * 100),
                       xytext=(5, 5), textcoords='offset points', fontsize=10)
        
        ax.set_title('Risk-Return Efficiency Frontier', 
                    fontsize=14, fontweight='bold')
        ax.set_xlabel('Annual Volatility (%)', fontsize=12)
        ax.set_ylabel('CAGR (%)', fontsize=12)
        ax.grid(alpha=0.3)
        ax.legend(loc='upper left', fontsize=11)
        
        plt.tight_layout()
        plt.show()
    
    @staticmethod
    def plot_metrics_table(metrics_dict):
        """Display performance metrics as table"""
        df = pd.DataFrame(metrics_dict).T
        
        # Format percentages
        pct_cols = ['Total Return', 'CAGR', 'Annual Volatility', 'Max Drawdown', 'Alpha']
        for col in pct_cols:
            if col in df.columns:
                df[col] = df[col].apply(lambda x: f'{x*100:.2f}%')
        
        # Format ratios
        ratio_cols = ['Sharpe Ratio', 'Sortino Ratio', 'Calmar Ratio', 'Beta']
        for col in ratio_cols:
            if col in df.columns:
                df[col] = df[col].apply(lambda x: f'{x:.3f}')
        
        fig, ax = plt.subplots(figsize=(12, 4))
        ax.axis('tight')
        ax.axis('off')
        
        table = ax.table(cellText=df.values,
                        rowLabels=df.index,
                        colLabels=df.columns,
                        cellLoc='center',
                        loc='center',
                        bbox=[0, 0, 1, 1])
        
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        table.scale(1, 2)
        
        # Color header
        for i in range(len(df.columns)):
            table[(0, i)].set_facecolor('#40466e')
            table[(0, i)].set_text_props(weight='bold', color='white')
        
        # Color row labels
        for i in range(len(df)):
            table[(i+1, -1)].set_facecolor('#40466e')
            table[(i+1, -1)].set_text_props(weight='bold', color='white')
        
        plt.title('Performance Metrics Summary', fontsize=14, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.show()


# ============================================================================
# 8. MAIN EXECUTION
# ============================================================================

def main():
    """Run complete analysis pipeline"""
    
    print("="*80)
    print("REGIME-BASED SECTOR ROTATION STRATEGY")
    print("Walk-Forward Implementation - No Look-Ahead Bias")
    print("="*80)
    print()
    
    # Step 1: Generate Data
    print("STEP 1: Generating Market Data...")
    generator = MarketDataGenerator()
    data = generator.generate()
    sectors = generator.sectors
    print(f"✓ Generated {len(data)} days of data for {len(sectors)} sectors\n")
    
    # Step 2: Engineer Features
    print("STEP 2: Engineering Features (T-1 Lag Enforced)...")
    features = FeatureEngine.engineer(data, sectors)
    print(f"✓ Created {len(features.columns)} features with {len(features)} observations\n")
    
    # Step 3: Detect Regimes
    print("STEP 3: Detecting Market Regimes (Walk-Forward)...")
    detector = RegimeDetector(n_regimes=3, min_history=252, retrain_freq=63)
    regimes, regime_probs = detector.detect_regimes(features)
    print(f"✓ Detected regimes: {regimes.value_counts().to_dict()}\n")
    
    # Step 4: Visualize Regime Detection
    print("STEP 4: Visualizing Regime Classification...")
    Visualizer.plot_regime_classification(data, regimes)
    Visualizer.plot_correlation_analysis(data, regimes, sectors)
    Visualizer.plot_hierarchical_clustering(data, regimes, sectors)
    
    # Step 5: Run Backtests
    print("\nSTEP 5: Running Strategy Backtests...")
    
    strategies = {
        'Conservative': 'conservative',
        'Institutional': 'institutional',
        'Aggressive': 'aggressive'
    }
    
    results_dict = {}
    
    for name, strategy_type in strategies.items():
        print(f"\n--- Running {name} Strategy ---")
        backtester = Backtester(
            data=data,
            features=features,
            regimes=regimes,
            sectors=sectors,
            initial_capital=100000,
            rebalance_freq=63,
            transaction_cost=0.001,
            strategy_type=strategy_type
        )
        results_dict[name] = backtester.run()
    
    # Step 6: Create Benchmark
    print("\nCreating Benchmark (S&P 500 Equal-Weight)...")
    benchmark = data[sectors].mean(axis=1)
    benchmark_curve = benchmark / benchmark.iloc[0] * 100000
    
    # Step 7: Calculate Metrics
    print("\nSTEP 6: Calculating Performance Metrics...")
    metrics_dict = {}
    
    for name, results in results_dict.items():
        metrics, dd = PerformanceAnalyzer.calculate_metrics(
            results['equity_curve'], 
            benchmark_curve
        )
        metrics_dict[name] = metrics
    
    # Benchmark metrics
    bench_metrics, bench_dd = PerformanceAnalyzer.calculate_metrics(benchmark_curve)
    metrics_dict['Benchmark'] = bench_metrics
    
    # Step 8: Visualize Results
    print("\nSTEP 7: Creating Performance Visualizations...\n")
    Visualizer.plot_equity_curves(results_dict, benchmark_curve)
    Visualizer.plot_drawdowns(results_dict, benchmark_curve)
    Visualizer.plot_weights_evolution(
        results_dict['Institutional']['weights_history'], 
        regimes
    )
    Visualizer.plot_risk_reward(metrics_dict)
    Visualizer.plot_metrics_table(metrics_dict)
    
    print("\n" + "="*80)
    print("ANALYSIS COMPLETE!")
    print("="*80)
    
    # Final summary
    print("\n📊 KEY FINDINGS:")
    print(f"   Benchmark Max DD: {bench_metrics['Max Drawdown']*100:.2f}%")
    print(f"   Institutional Max DD: {metrics_dict['Institutional']['Max Drawdown']*100:.2f}%")
    print(f"   Drawdown Reduction: {(1 - metrics_dict['Institutional']['Max Drawdown']/bench_metrics['Max Drawdown'])*100:.1f}%")
    print(f"\n   Benchmark Sharpe: {bench_metrics['Sharpe Ratio']:.3f}")
    print(f"   Institutional Sharpe: {metrics_dict['Institutional']['Sharpe Ratio']:.3f}")
    print(f"   Sharpe Improvement: {((metrics_dict['Institutional']['Sharpe Ratio']/bench_metrics['Sharpe Ratio'])-1)*100:.1f}%")
    print("\n✅ No look-ahead bias - all signals generated using historical data only")
    print("✅ Rebalanced every 63 days based on detected regimes")
    print("✅ Transaction costs included (0.10% per trade)\n")


if __name__ == "__main__":
    main()